自由財經

In [13]:
#################################################################################
# 更改
# stock_id = '0000'
# glob_url = https://search.ltn.com.tw/list?keyword=股票中文名稱&start_time=20041201&end_time=20240402&sort=date&type=all&page=
 ################################################################################

stock_id = "3443"
global_url = 'https://search.ltn.com.tw/list?keyword=創意&start_time=20041201&end_time=20240402&sort=date&type=all&page='

In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

page = 1
news_url_l = []

for i in range(10):
    url =  global_url + str(i+1)
    
    # 創建 WebDriver 實例
    driver = webdriver.Chrome()
    
    # 設置隱式等待時間
    driver.implicitly_wait(10) # seconds
    
    # 載入網頁
    driver.get(url)
    
    try:
        # 使用顯式等待，等待特定元素出現
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "cont")]'))
        )
        
        # 獲取所有元素
        elements = driver.find_elements(By.XPATH, '//div[contains(@class, "cont")]')
        
        for e in elements:
            # 獲取該元素的 href 屬性
            href = e.get_attribute('href')
            if href:
                print("目標元素的網址:", href)
                news_url_l.append(href)
                
    except TimeoutException:
        print("網頁載入超時，重新載入...")
        # 關閉目前的 WebDriver 實例
        driver.quit()
        continue
    
    # 關閉 WebDriver 實例
    driver.quit()


目標元素的網址: https://ec.ltn.com.tw/article/breakingnews/4627762
目標元素的網址: https://news.ltn.com.tw/news/politics/breakingnews/4627605
目標元素的網址: https://playing.ltn.com.tw/article/29260
目標元素的網址: https://news.ltn.com.tw/news/life/breakingnews/4627593
目標元素的網址: https://news.ltn.com.tw/news/life/breakingnews/4627429
目標元素的網址: https://ec.ltn.com.tw/article/breakingnews/4627542
目標元素的網址: https://news.ltn.com.tw/news/novelty/breakingnews/4627358
目標元素的網址: https://ec.ltn.com.tw/article/breakingnews/4627031
目標元素的網址: https://food.ltn.com.tw/article/12130
目標元素的網址: https://ec.ltn.com.tw/article/breakingnews/4627092
目標元素的網址: https://playing.ltn.com.tw/article/29256
目標元素的網址: https://istyle.ltn.com.tw/article/30343
目標元素的網址: https://news.ltn.com.tw/news/life/paper/1638808
目標元素的網址: https://news.ltn.com.tw/news/society/paper/1638789
目標元素的網址: https://news.ltn.com.tw/news/politics/breakingnews/4626960
目標元素的網址: https://art.ltn.com.tw/article/breakingnews/4626922
目標元素的網址: https://art.ltn.com.tw/article/breakingnews/46

In [15]:
# 找到所有自由財經的網址
news_url_l2 = []
for href in news_url_l:
    # 檢查網址是否以 "ec" 開頭
    if href.startswith("https://ec"):
        news_url_l2.append(href)
news_url_l2 = news_url_l2[:]

In [16]:
import time
import os
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import re

content = ''
date_last = ''
article = ''

for index, link in enumerate(news_url_l2):
    # 請求網頁內容
    response = requests.get(link)
    # 解析網頁內容
    soup = BeautifulSoup(response.content, 'html.parser')

    try:
        time_element = soup.find_all('span', class_='time')
        time_text = time_element[1].text.strip()
        time_datetime = datetime.strptime(time_text, '%Y/%m/%d %H:%M')
        formatted_date = time_datetime.strftime('%Y%m%d')
        # print(formatted_date)
        
        if date_last != formatted_date:
            article = ''
        date_last = formatted_date
        
        p = soup.find_all('div', class_="text")[1]
        contents = p.find_all('p')
        # contents = p1.find_all('p')
        
        
        for content in contents:
            article = article + content.text.strip()
            
        # print(article)
        
        folder_path = "stock_news/"+stock_id+"/"+stock_id+"_news_ltn/"
        file_name = stock_id+"n_" + formatted_date
        if not os.path.exists(os.path.dirname(folder_path)):
            print("建立資料夾" + folder_path)
            os.makedirs(os.path.dirname(folder_path))
        
        with open(folder_path + file_name, "w" , encoding='UTF-8') as f:
            f.write(article)      
            
        print(f'[{index+1}/{len(news_url_l2)}], Date:{formatted_date}, link:{link}')
        # print('日期', content + "寫入", folder_path + file_name)
            
    except Exception as e:
        print("爬取失敗", e)
        print(f'[{index+1}/{len(news_url_l2)}], Date:{formatted_date}, link:{link}')
        pass
    time.sleep(1)


建立資料夾stock_news/3443/3443_news_ltn/
[1/9], Date:20240402, link:https://ec.ltn.com.tw/article/breakingnews/4627762
[2/9], Date:20240402, link:https://ec.ltn.com.tw/article/breakingnews/4627542
[3/9], Date:20240402, link:https://ec.ltn.com.tw/article/breakingnews/4627031
[4/9], Date:20240402, link:https://ec.ltn.com.tw/article/breakingnews/4627092
[5/9], Date:20240401, link:https://ec.ltn.com.tw/article/breakingnews/4626679
[6/9], Date:20240401, link:https://ec.ltn.com.tw/article/breakingnews/4626082
[7/9], Date:20240327, link:https://ec.ltn.com.tw/article/breakingnews/4621529
[8/9], Date:20240324, link:https://ec.ltn.com.tw/article/breakingnews/4617674
[9/9], Date:20240320, link:https://ec.ltn.com.tw/article/breakingnews/4614441


In [17]:
# import time
# import re
# from datetime import datetime
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# import os

# #設定重新執行的次數
# max_retries = 3
# p = ""
# date_last = ''

# def load_website(link):
#     retries = 0
#     while retries < max_retries:
#         driver = webdriver.Chrome()
#         driver.get(link)
#         driver.set_page_load_timeout(5)
#         # 設置隱式等待時間（秒）
#         driver.implicitly_wait(5)
#         try:
#             #time
#             time_element = driver.find_element(By.XPATH,'//*[@id="ec"]/div[7]/section/div[4]/div[1]/span')
#             time_text = time_element.text
#             date_time_obj = datetime.strptime(time_text, '%Y/%m/%d %H:%M')
#             formatted_date = date_time_obj.strftime('%Y%m%d')
#             print(formatted_date)
            
#             # 文章內容
#             if date_last != formatted_date:
#                 p = ''
            
#             paragraphs = driver.find_elements(By.XPATH, "//div[@class='text']//p")
#             # 提取每個 <p> 標籤的文本
#             # print(paragraphs)
#             for paragraph in paragraphs:
#                 text = paragraph.text
#                 p = p + text
            
#             folder_path = "stock_news/"+stock_id+"/"+stock_id+"_news_ltn/"
#             file_name = stock_id+"n_" + formatted_date
#             if not os.path.exists(os.path.dirname(folder_path)):
#                 print("建立資料夾" + folder_path)
#                 os.makedirs(os.path.dirname(folder_path))
            
#             with open(folder_path + file_name, "w") as f:
#                 f.write(p)
            
#             driver.quit()
#             return  # 成功載入，退出函數
#         except Exception as e:
#             retries += 1
#             print(f"載入失敗{retries}次")
#             print(f"錯誤信息：{e}")
#             driver.quit()
    
#     print(f"連續 {max_retries} 次載入失敗，跳過該網站")
#     time.sleep(1)

# # 要加載的網站列表
# for index, link in enumerate(news_url_l2):
#     print(f'[{index+1}/{len(news_url_l2)}], link:{link}')
#     load_website(link)


In [18]:
#test cell
# driver = webdriver.Chrome()
# driver.get('https://ec.ltn.com.tw/article/breakingnews/4612859')
# driver.set_page_load_timeout(5)
# # 設置隱式等待時間（秒）
# driver.implicitly_wait(5)  # 設置為10秒
# 
# p = ""
# try:
#     #time
#     time_element = driver.find_element(By.XPATH,'//*[@id="ec"]/div[7]/section/div[4]/div[1]/span')
#     time_text = time_element.text
#     
#     # 將日期時間字串解析為 datetime 物件
#     date_time_obj = datetime.strptime(time_text, '%Y/%m/%d %H:%M')
# 
#     # 提取日期部分並格式化為指定格式
#     formatted_date = date_time_obj.strftime('%Y%m%d')
#     print(formatted_date)
#     
#     # 文章內容
#     paragraphs = driver.find_elements(By.XPATH, "//div[@class='text']//p")
#     # 提取每個 <p> 標籤的文本
#     # print(paragraphs)
#     p = ""
#     for paragraph in paragraphs:
#         text = paragraph.text
#         p = p + text
# except Exception as e:
#     print(e)
#     
# print(p)
# driver.quit()